#### Importation des bibliothèques nécessaires

In [1]:
from minio import Minio  # Minio pour interagir avec le stockage S3 (MinIO).
import pandas as pd  # Pandas pour manipuler les données sous forme de DataFrame.
import s3fs  # S3FS pour interagir avec MinIO via le système de fichiers.
import psycopg2  # psycopg2 pour se connecter à la base de données PostgreSQL.
import os

#### Paramètres de connexion à la base de données PostgreSQL

In [2]:
db_params = {
    'dbname': 'inflation_db',  # Nom de la base de données.
    'user': 'postgres',  # Utilisateur de la base de données.
    'password': 'postgrespassword',  # Mot de passe pour l'utilisateur.
    'host': 'localhost',  # Adresse de l'hôte.
    'port': 5432  # Port.
}

#### Nom de la table à traiter

In [3]:
table_name = "inflation_data"

#### Fonction pour établir la connexion à la base de données PostgreSQL


In [4]:
def get_conn():
    conn = psycopg2.connect(**db_params)
    return conn

#### Établir la connexion à la base de données

In [5]:
conn = get_conn()

#### Requête SQL pour récupérer toutes les données de la table

In [6]:
query = f"SELECT * FROM {table_name}"

#### Charger les données de la table PostgreSQL dans un DataFrame Pandas

In [7]:
raw_df = pd.read_sql(query, conn)

C:\Users\SAMBA\AppData\Local\Temp\ipykernel_22112\2253497135.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  raw_df = pd.read_sql(query, conn)


#### Affichage du DataFrame brut

In [8]:
raw_df

,country,country_code,year,inflation
0,Afghanistan,AFG,2001,-11.774536
1,Afghanistan,AFG,2002,6.125522
2,Afghanistan,AFG,2003,11.655238
3,Afghanistan,AFG,2004,11.271432
4,Afghanistan,AFG,2005,10.912774
...,...,...,...,...
13772,Zimbabwe,ZWE,2019,225.394648
13773,Zimbabwe,ZWE,2020,604.945864
13774,Zimbabwe,ZWE,2021,113.294981
13775,Zimbabwe,ZWE,2022,266.987152


### Étape 1 : Nettoyage des données
#### Vérifier s'il y a des valeurs manquantes

In [9]:
print("Nombre de valeurs manquantes avant nettoyage :")
print(raw_df.isnull().sum())

Nombre de valeurs manquantes avant nettoyage :
country         0
country_code    0
year            0
inflation       0
dtype: int64


#### Suppression des lignes contenant des valeurs manquantes

In [10]:
raw_df = raw_df.dropna()

#### Vérification du nettoyage


In [11]:
print("Nombre de valeurs manquantes après nettoyage :")
print(raw_df.isnull().sum())

Nombre de valeurs manquantes après nettoyage :
country         0
country_code    0
year            0
inflation       0
dtype: int64


### Étape 2 : Transformation des données
#### Vérifier les types des colonnes

In [12]:
print("Types des colonnes avant transformation :")
print(raw_df.dtypes)

Types des colonnes avant transformation :
country          object
country_code     object
year              int64
inflation       float64
dtype: object


#### S'assurer que les colonnes ont le bon type

In [13]:
raw_df['year'] = raw_df['year'].astype(int)
raw_df['inflation'] = raw_df['inflation'].astype(float)

#### Vérification des types après transformation

In [14]:
print("Types des colonnes après transformation :")
print(raw_df.dtypes)

Types des colonnes après transformation :
country          object
country_code     object
year              int32
inflation       float64
dtype: object


### Étape 3 : Sauvegarder le DataFrame traité en CSV

In [15]:
output_csv = "data/processed_inflation_data.csv"
raw_df.to_csv(output_csv, index=False)

### Étape 4 : Sauvegarder dans MinIO
#### Connexion à MinIO

In [16]:
minio_client = Minio(
    "localhost:9000",
    access_key="minioadmin",
    secret_key="minioadmin",
    secure=False
)

#### Vérifier si le bucket existe, sinon le créer

In [17]:
bucket_name = "inflation-bucket"
if not minio_client.bucket_exists(bucket_name):
    minio_client.make_bucket(bucket_name)
    print(f"Bucket '{bucket_name}' créé avec succès.")
else:
    print(f"Le bucket '{bucket_name}' existe déjà.")


Le bucket 'inflation-bucket' existe déjà.


#### Téléverser le fichier CSV dans MinIO

In [18]:
minio_client.fput_object(bucket_name, "processed_inflation_data.csv", output_csv)
print(f"Fichier '{output_csv}' chargé avec succès dans MinIO.")

Fichier 'data/processed_inflation_data.csv' chargé avec succès dans MinIO.
